# Question 1: Supervised Learning for Bioprinting

In this question, we continue creating machine learning algorithms which can predict cell viability from
bioprinting parameters, similar to Shuyu Tian et al. “Machine assisted experimentation of extrusion-based bioprinting systems”. In: *Micromachines* 12.7 (2021), p. 780. However, this time we are using neural networks.

You will train in PyTorch:

• a classification neural network to predict acceptable/unacceptable cell viability,

• a regression neural network to predict real-valued cell viability.



Preprocess the dataset according to the provided code in this thesis. https://scholarscompass.vcu.edu/cgi/viewcontent.cgi?article=7979&context=etd

For classification, do not use the “Acceptable Viability?” column. Instead, label a sample as acceptable if
viability ≥ 70%, and unacceptable otherwise. Split both datasets 85/15 into training and test sets.

Validation protocol: On the training set, perform 5-fold cross validation. Report the mean performance
across folds and the performance when training on the full training set.

Metrics:
• Regression: MSE
• Classification: Accuracy, Precision, Recall



Important: Only evaluate the default model and your final best model on the test set.
1. Default models. Train regression and classification networks in PyTorch using the default optimizer
settings from Tutorial 2. Report training (5-fold CV + full training) and test performance.
2. Activation functions. Test ReLU and tanh, plus one mixed-activation configuration. State which
performs best.
3. Learning rate and momentum. Try two learning rates from different orders of magnitude and train
with/without momentum. Plot and discuss the effect on the 5 training loss curves.
4. Best model vs default. Combine your best hyperparameters, train on the full training set, and
evaluate on the test set. Compare to the default model and discuss.

In [6]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import torch
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import warnings
from scipy.integrate import odeint
from matplotlib import style
import numpy as np

warnings.filterwarnings('ignore')

In [7]:
# CUDA support
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f"Device is {device}")

Device is cpu


### Import data

In [ ]:
import pickle

with open("A2Q1_data.pkl", "rb") as f:
    ds_classification = pickle.load(f)

with open("A2Q1_data_regression.pkl", "rb") as f:
    ds_regression = pickle.load(f)

def to_tensors(data):
    X_train = torch.tensor(data["X_train"].astype(np.float32).values, dtype=torch.float32)
    X_test  = torch.tensor(data["X_test"].astype(np.float32).values,  dtype=torch.float32)
    y_train = torch.tensor(data["y_train"].values, dtype=torch.float32).unsqueeze(1)
    y_test  = torch.tensor(data["y_test"].values,  dtype=torch.float32).unsqueeze(1)
    return X_train, X_test, y_train, y_test

X_train_c, X_test_c, y_train_c, y_test_c = to_tensors(ds_classification)
X_train_r, X_test_r, y_train_r, y_test_r = to_tensors(ds_regression)

INPUT_DIM = X_train_c.shape[1] # 45

### 5-Fold CV:

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)

fold_metrics = []
fold_losses = []

for fold, (training_index, validation_index) in enumerate(kf.split(X_train_c)):
    